## Notebook specific deps

In [ ]:
!uv pip install kagglehub seaborn scikit-learn imblearn safetensors jupyter

## Download the dataset

In [ ]:
import shutil
from pathlib import Path

import kagglehub

DATASET_PATH = Path("./dataset/pima-indians-diabetes-database")

if not DATASET_PATH.exists():
    # Download latest version
    path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")
    shutil.move(path, DATASET_PATH)
else:
    print(f"{DATASET_PATH} exists")

## Dataset Understanding

In [ ]:
import pandas as pd

column_names = [
    "Pregnancies",
    "Glucose",
    "BloodPressure",
    "SkinThickness",
    "Insulin",
    "BMI",
    "DiabetesPedigreeFunction",
    "Age",
    "y",
]
df_diabetes = pd.read_csv(DATASET_PATH / "diabetes.csv", header=0, names=column_names)

In [ ]:
df_diabetes.info()

In [ ]:
df_diabetes

In [ ]:
df_diabetes.hist(figsize=(9, 9))

#### Target value counts

In [ ]:
df_diabetes["y"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(x="y", data=df_diabetes)
plt.show()

In [ ]:
# df_diabetes['y'].value_counts().plot(kind='pie')
plt.pie(df_diabetes["y"].value_counts().tolist(), labels=["0", "1"], autopct="%1.1f%%")

plt.title("Outcomes")
plt.tight_layout()

plt.show()

#### Diabetes % by age range

In [ ]:
# Define Age ranges
age_bins = [20, 30, 40, 50, 60, 70, 80]  # Adjust the age ranges as needed
age_labels = ["20-29", "30-39", "40-49", "50-59", "60-69", "70+"]

# Use the pd.cut function to categorize ages into age ranges:
df_diabetes["AgeRange"] = pd.cut(
    df_diabetes["Age"], bins=age_bins, labels=age_labels, include_lowest=True
)

# Group the data by age range and calculate the percentage of diabetes cases (Diabetes=1):
age_diabetes = df_diabetes.groupby("AgeRange", observed=True)["y"].mean() * 100

# Plot the data:
plt.bar(age_diabetes.index, age_diabetes)
plt.xlabel("Age Range")
plt.ylabel("Percentage with Diabetes")
plt.title("Diabetes Percentage by Age Range")
plt.show()

#### BMI vs. Diabetes

In [ ]:
# Define BMI ranges
bmi_ranges = [0, 18.5, 24.9, 29.9, 34.9, 100]
bmi_labels = ["Underweight", "Normal", "Overweight", "Obese (I)", "Obese (II+)"]

# Use the pd.cut function to categorize ages into age ranges:
df_diabetes["BMIRange"] = pd.cut(
    df_diabetes["BMI"], bins=bmi_ranges, labels=bmi_labels, include_lowest=True
)

# Group the data by age range and calculate the percentage of diabetes cases (Diabetes=1):
bmi_diabetes = df_diabetes.groupby("BMIRange", observed=True)["y"].mean() * 100

# Plot the data:
plt.bar(bmi_diabetes.index, bmi_diabetes)
plt.xlabel("BMI Range")
plt.ylabel("Percentage with Diabetes")
plt.title("Diabetes Percentage by BMI Range")
plt.show()

#### Glucose vs Diabetes

In [ ]:
# Create two separate data series for diabetic and non-diabetic patients:
glucose_diabetic = df_diabetes[df_diabetes["y"] == 1]["Glucose"]

glucose_non_diabetic = df_diabetes[df_diabetes["y"] == 0]["Glucose"]
# Define the bins (ranges) for the histogram
bins = [50, 70, 100, 125, 150, 200, 250]

# Plot histogram for diabetic patients
plt.hist(glucose_diabetic, bins=bins, alpha=0.5, label="Diabetic", color="red")

# Plot histogram for non-diabetic patients
plt.hist(glucose_non_diabetic, bins=bins, alpha=0.5, label="Non-Diabetic", color="blue")

# Labeling and legend
plt.xlabel("Glucose Level")
plt.ylabel("Frequency")
plt.title("Glucose Distribution by Diabetes Status")
plt.legend()

# Show the plot
plt.show()

## Data Cleaning

In [ ]:
# Missing value percentage checking
missing_percentage = (df_diabetes.isnull().sum() / len(df_diabetes)) * 100

# Show the missing percentage
print(missing_percentage)

#### Drop zero values

In [ ]:
# Exclude the 'y' column from the selection
numeric_columns = df_diabetes.select_dtypes(include=["number"]).drop(columns=["y"])

# Count the number of zeros in each selected column
zero_counts = numeric_columns.eq(0).sum()

# Print the columns with zero counts
print("Columns with zero values and their counts (excluding 'y'):")
print(zero_counts[zero_counts > 0])

Since `SkinThickness` and `Insulin` has many zeros, we drop them

In [ ]:
columns_to_drop = ["SkinThickness", "Insulin", "AgeRange", "BMIRange"]
df_new = df_diabetes.drop(columns_to_drop, axis=1)

In [ ]:
df_new.info()

In [ ]:
df_new

#### Treating the zero values in the rest of the columns

We will replace the 0 values with mean / median value

In [ ]:
# Calculate the skewness for all numerical columns
skewness = df_new.skew()

# Print the skewness values
print(skewness)

In [ ]:
# Calculate mean and median (excluding zeros)
mean_glucose = df_new[df_new["Glucose"] != 0]["Glucose"].mean()
median_bmi = df_new[df_new["BMI"] != 0]["BMI"].median()
median_bp = df_new[df_new["BloodPressure"] != 0]["BloodPressure"].median()

In [ ]:
# Replace zeros across multiple columns at once
df_new.replace(
    {
        "Glucose": {0: mean_glucose},
        "BMI": {0: median_bmi},
        "BloodPressure": {0: median_bp},
    },
    inplace=True,
)

In [ ]:
# Exclude the 'y' column from the selection
numeric_columns = df_new.select_dtypes(include=["number"]).drop(columns=["y"])

# Count the number of zeros in each selected column
zero_counts = numeric_columns.eq(0).sum()

# Print the columns with zero counts
print("Columns with zero values and their counts (excluding 'y'):")
print(zero_counts[zero_counts > 0])

## Train - Test Data Split and Processing

In [ ]:
df_new

In [ ]:
features = df_new.values[:, :6]
print(features.shape)

In [ ]:
labels = df_new.values[:, 6:]
print(labels.shape)

In [ ]:
# Split your dataset into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=95
)

In [ ]:
print(f"{X_train.shape = }, {y_train.shape = }")
print(f"{X_test.shape = }, {y_test.shape = }")

#### Apply SMOTE to the training data

SMOTE is a statistical technique for dealing with imbalanced datasets, since we have an imbalance between diabetic and non-diabetic cases (~35% vs ~65% as shown in the pie chart). What SMOTE does:
- Identifies minority class samples: In this case, patients with diabetes (class 1)
- Creates synthetic examples: Instead of simply duplicating minority samples, SMOTE:
    - Selects a minority class sample
    - Finds its k-nearest neighbors (other minority samples)
    - Generates new synthetic samples along the lines connecting the sample and its neighbors
    - Balances the dataset: The resulting training data (`X_train_resampled`, `y_train_resampled`) has equal representation of both classes

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"{X_train_resampled.shape = }, {y_train_resampled.shape = }")

In [ ]:
import numpy as np

num_ones = np.sum(y_train_resampled == 1)
num_zeros = np.sum(y_train_resampled == 0)

print(f"Number of 1s: {num_ones}")
print(f"Number of 0s: {num_zeros}")

#### Scale the data
Transforms each feature (column) using: `(x - mean) / std_dev`  
Benefits:
- Equalizes feature scales: Features like Glucose (70-200) and Pregnancies (0-17) are brought to the same scale
- Improves neural network performance: The model won't be biased toward features with larger values
- Speeds up gradient descent: Optimization converges faster with standardized data
- Reduces sensitivity to outliers: Using unit variance makes the model more robust

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

In [ ]:
print(f"{X_train_resampled.shape = }, {y_train_resampled.shape = }")
print(f"{X_test.shape = }, {y_test.shape = }")

## Model Training

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from loguru import logger
from safetensors.torch import load_file, save_file
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.backends.xla.is_available():
        return torch.device("xla")
    elif torch.backends.xpu.is_available():
        return torch.device("xpu")
    elif torch.backends.ipu.is_available():
        return torch.device("ipu")
    elif torch.backends.meta.is_available():
        return torch.device("meta")
    else:
        return torch.device("cpu")


DEVICE = get_device()
logger.info(f"Using device: {DEVICE}")

#### DataLoader

In [ ]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_resampled)
y_train_tensor = torch.FloatTensor(y_train_resampled).reshape(-1, 1)  # Add this reshape
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=len(test_dataset), shuffle=False
)

#### Model Architecture

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim=6):
        super(Net, self).__init__()
        # First layer with more units and batch normalization
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 32),  # Increased from 20 to 32
            nn.BatchNorm1d(32),  # Added batch normalization
            nn.LeakyReLU(0.1),  # LeakyReLU instead of ReLU
            nn.Dropout(0.2),  # Increased dropout
        )

        # Second layer with more units
        self.layer2 = nn.Sequential(
            nn.Linear(32, 24),  # Increased from 14 to 24
            nn.BatchNorm1d(24),  # Added batch normalization
            nn.LeakyReLU(0.1),
            nn.Dropout(0.25),
        )

        # Third layer
        self.layer3 = nn.Sequential(
            nn.Linear(24, 16), nn.BatchNorm1d(16), nn.LeakyReLU(0.1)
        )

        # Output layer
        self.output_layer = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output_layer(x)
        return x


# Initialize the improved model with weight decay
model = Net().to(DEVICE)
criterion = nn.BCELoss()

# Improved optimizer settings
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,  # Increased learning rate from 0.0001
    weight_decay=0.0005,  # Added L2 regularization
)

# Add learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.5,
    patience=10,
)

#### Train and Eval

In [ ]:
Path("./weights").mkdir(parents=True, exist_ok=True)
WEIGHT_PATH = Path("./weights/local_training_diabetes_model.safetensors")


# Training function
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=100):
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    best_val_acc = 0.0

    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        history["train_loss"].append(epoch_loss)
        history["train_acc"].append(epoch_acc)

        # Validation phase
        val_loss, val_acc = evaluate_model(model, test_loader, criterion)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        scheduler.step(val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {epoch_loss:.4f} | "
            f"Train Acc: {epoch_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"Val Acc: {val_acc:.4f}"
        )

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            save_file(model.state_dict(), WEIGHT_PATH)
            print(
                f"\n Find best model with val_acc {best_val_acc}. saved model to {WEIGHT_PATH}"
            )

    return history

In [ ]:
# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct / total

    return epoch_loss, epoch_acc

In [ ]:
# Train the model
history = train_model(model, train_loader, test_loader, criterion, optimizer)

In [ ]:
# Create a new model instance
loaded_model = Net().to(DEVICE)

# Load the weights from SafeTensors file
loaded_model.load_state_dict(load_file(WEIGHT_PATH))

# Evaluate the loaded model
test_loss, test_acc = evaluate_model(loaded_model, test_loader, criterion)
print(f"Loaded model - Test loss: {test_loss:.4f}, Test accuracy: {test_acc*100:.2f}%")